In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

In [2]:
from pyspark.sql.types import Row
from datetime import datetime

In [3]:
airlines = spark.read.format('csv').option('header', True).load('datasets/airlines.csv')
airports = spark.read.format('csv').option('header', True).load('datasets/airports.csv')
flights = spark.read.format('csv').option('header', True).load('datasets/flights.csv')

In [8]:
airlines.createOrReplaceTempView("airlines")
airports.createOrReplaceTempView("airports")
flights.createOrReplaceTempView("flights")

In [7]:
spark.sql('select * from airlines').show(5)

+-----+--------------------+
| Code|         Description|
+-----+--------------------+
|19031|Mackey Internatio...|
|19032|Munz Northern Air...|
|19033|Cochise Airlines ...|
|19034|Golden Gate Airli...|
|19035|  Aeromech Inc.: RZZ|
+-----+--------------------+
only showing top 5 rows



In [9]:
flights.count(), airlines.count(), airports.count()

(476881, 1579, 6323)

In [10]:
spark.sql("select * from airports").show(5)

+----+--------------------+
|Code|         Description|
+----+--------------------+
| 01A|Afognak Lake, AK:...|
| 03A|Granite Mountain,...|
| 04A|Lik, AK: Lik Mini...|
| 05A|Little Squaw, AK:...|
| 06A|Kizhuyak, AK: Kiz...|
+----+--------------------+
only showing top 5 rows



In [11]:
spark.sql("select * from flights").show(5)

+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|      date|airlines|flight_number|origin|destination|departure|departure_delay|arrival|arrival_delay|air_time|distance|
+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|2014-04-01|   19805|            1|   JFK|        LAX|     0854|          -6.00|   1217|         2.00|  355.00| 2475.00|
|2014-04-01|   19805|            2|   LAX|        JFK|     0944|          14.00|   1736|       -29.00|  269.00| 2475.00|
|2014-04-01|   19805|            3|   JFK|        LAX|     1224|          -6.00|   1614|        39.00|  371.00| 2475.00|
|2014-04-01|   19805|            4|   LAX|        JFK|     1240|          25.00|   2028|       -27.00|  264.00| 2475.00|
|2014-04-01|   19805|            5|   DFW|        HNL|     1300|          -5.00|   1650|        15.00|  510.00| 3784.00|
+----------+--------+-----------

In [17]:
total_dist_travelled = spark.sql("select * from flights") \
    .groupBy('airlines')\
    .agg({'distance': 'sum'})\
    .withColumnRenamed('sum(distance)', 'total dist travelled') \
    .sort('total dist travelled', ascending=False) 
total_dist_travelled.show(5)

+--------+--------------------+
|airlines|total dist travelled|
+--------+--------------------+
|   19393|         7.0116607E7|
|   19790|         5.7091355E7|
|   19977|         5.1629879E7|
|   19805|         4.6572644E7|
|   20355|         3.0257721E7|
+--------+--------------------+
only showing top 5 rows



In [21]:
total_dist_travelled_sql = spark.sql("""
    select airlines, SUM(distance) as total_dist_travelled from flights
    GROUP BY airlines ORDER BY total_dist_travelled desc
""")
total_dist_travelled_sql.show(5)

+--------+--------------------+
|airlines|total_dist_travelled|
+--------+--------------------+
|   19393|         7.0116607E7|
|   19790|         5.7091355E7|
|   19977|         5.1629879E7|
|   19805|         4.6572644E7|
|   20355|         3.0257721E7|
+--------+--------------------+
only showing top 5 rows



In [31]:
total_dist_travelled_sql = spark.sql("""
    select flights.airlines, SUM(distance) as total_dist_travelled, Description
    from flights JOIN airlines
    ON airlines.Code = flights.airlines
    GROUP BY airlines, Description ORDER BY total_dist_travelled desc
""")
total_dist_travelled_sql.show(5)

+--------+--------------------+--------------------+
|airlines|total_dist_travelled|         Description|
+--------+--------------------+--------------------+
|   19393|         7.0116607E7|Southwest Airline...|
|   19790|         5.7091355E7|Delta Air Lines I...|
|   19977|         5.1629879E7|United Air Lines ...|
|   19805|         4.6572644E7|American Airlines...|
|   20355|         3.0257721E7| US Airways Inc.: US|
+--------+--------------------+--------------------+
only showing top 5 rows

